<a href="https://colab.research.google.com/github/zaira-ra/ENSF444_Project/blob/main/ENSF444Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Add some sort of introduction text]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load Data